# 1
Passar um dado FOTS, tendo as seguintes informações:
* init(s)
* inv(s)
* trans(s,v)
* exit(s)

para um SMT


a)

In [ ]:
def init(s):
    return True

def inv(s):
    return True

def trans(u,v):
    return True

def exit(s):
    return True

b)  

Tendo em conta que:
1. $init(s)\rightarrow inv(s)$
2. $trans(l,x,l'x') \wedge inv(l)\rightarrow inv(l')$

são tautologias.

De forma equivalente, e mais adequado ao SMT, podemos provar que as suas formas negadas:
1. $init(s) \wedge \neg inv(s)$ $\hspace{4cm}$(non rooted)
2. $trans(l,x,l'x') \wedge inv(l)\wedge \neg inv(l')$ $\hspace{1cm}$(non_safe)

são insatisfazíveis (i.e. *unsat*).

c)  
1-induction

k-induction

# 2
Passar codigo de Euclides para FOTS

## Codificar o algoritmo de Euclides

0: $assert\quad 0\quad and\quad b > 0$  
1: $while\quad a > 0\quad and\quad b > 0:$  
2: $\hspace{1cm}if\quad b > a$  
3: $\hspace{2cm}b = b - a$  
4: $\hspace{1cm}else:$  
5: $\hspace{2cm}a = a - b$    
6: $pass$   


* **var1** e **var2** representam respetivamente a e b do código acima;  
* **place** representa o numero da linha do codigo acima;        
* **N** representa o numero de linhas usadas no algoritmo acima;  

In [1]:

from z3 import *

Modes, (var1, var2, place) = EnumSort('Modes', ['var1', 'var2', 'place'])
Estado = ArraySort(Modes, IntSort())

N = 7

s = Const('s', Estado)
w = Const('w', Estado)


#### inv

In [2]:
def inv(s):
  l = s[place]
  return(l in range(N))


#### init


In [3]:
def init(s):
  l = s[place]
  return( l==0 )

#### trans

In [4]:
def trans(u,v):
  #(a, b, l) = u 
  a = u[var1]; b = u[var2]; l = u[place];
  #(a_, b_, l_) = v
  a_ = v[var1]; b_ = v[var2]; l_ = v[place];
  return Or(
              And(l == 0 , l_ == 1 , a > 0 , b > 0 , a_ == a , b_ == b),
              And(l == 1 , l_ == 2 , a > 0 , b > 0 , a_ == a , b_ == b),
              And(l == 2 , l_ == 3 , a > 0 , b > a , a_ == a , b_ == b),
              And(l == 3 , l_ == 1 , a > 0 , b > a , a_ == a , b_ == b - a),
              And(l == 2 , l_ == 4 , a > 0 , b > 0 , a_ == a , b_ == b),
              And(l == 4 , l_ == 5 , a >= b , b > 0 , a_ == a , b_ == b),
              And(l == 5 , l_ == 1 , a >= b , b > 0 , a_ == a - b , b_ == b),
              And(l == 1 , l_ == 6 , a == 0 , b > 0 , a_ == a , b_ == b)
            )


#### Exit

In [5]:
def exit(s):
    l = s[place]
    return (l == 6)